# Эмбеддинги (векторные представления) слов

### **1. Что такое word embedding?**

При работе с текстами нам хотелось бы понимать, как соотносятся между собой слова в языке (например, насколько они похожи или различны — в каком-то релевантном для решаемой задачи смысле). Поэтому мы можем работать не с самими словами в формате строк, а с их числовым — или скорее векторным — представлением.

Один из очевидных способов представить слово в виде строки — one-hot encoding. Однако такое представление, с одной стороны, требует огромного ресурса памяти (например, в русском языке более 100000 слов), а, с другой стороны, не даёт содержательной информации о том, как эти слова между собой соотносятся.

### **2. Word2Vec**

Краткое напоминание о том, что такое косинусное расстояние между векторами $x$ и $y$, которое можно рассматривать как меру сходства между ними:

$similarity(x, y) = \cos(\Theta) = \frac{\langle x, y \rangle}{||x|| \cdot ||y||}$, где $\Theta$ — угол между векторами.

Модель Word2Vec строит такие векторные представления, чтобы векторы похожих слов оказывались близки по косинусному расстоянию. Похожими считаются слова, которые часто встречаются в одном и том же контексте.

Есть различные подходы к тому, как обучать векторные представления. Первый из них — CBoW (Continuous Bag of Words), предсказание слова по контексту, и второй — SkipGram, предсказание контекста по слову. Подробнее об архитектуре и модификациях функции потерь можно прочитать [здесь](https://https://arxiv.org/pdf/1301.3781.pdf) и [здесь](https://https://arxiv.org/abs/1310.4546).

Рассмотрим word2vec, [реализованный](https://https://radimrehurek.com/gensim/models/word2vec.html) в библиотеке gensim.

In [1]:
!pip install -U gensim

     |████████████████████████████████| 24.2MB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 1.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 992kB/s eta 0:00:01
     |████████████████████████████████| 5.4MB 498kB/s eta 0:00:01
     |████████████████████████████████| 71kB 2.0MB/s eta 0:00:01
     |████████████████████████████████| 552kB 2.4MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.9.0-cp37-none-any.whl size=73088 sha256=679c74db191962bcabd5e564cfbf3f0d450d871a0dda82c13eea12b1a511e78a
  Stored in directory: /home/sorain/.cache/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open


Посмотрим, какие доступны предобученные модели.

In [2]:
from gensim.models import Word2Vec
import gensim.downloader as api

api.info()

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1,
   'record_format': 'dict',
   'file_size': 6344358,
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py',
   'license': 'All files released for the task are free for general research use',
   'fields': {'2016-train': ['...'],
    '2016-dev': ['...'],
    '2017-test': ['...'],
    '2016-test': ['...']},
   'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.',
   'checksum': '701ea67acd82e75f95e1d8e62fb0ad29',
   'file_name': 'se

Среди них есть word2vec-ruscorpora-300 — word2vec, обученный на Национальном корпусе русского языка. Можно выгрузить эту модель и, например, посмотреть на слова, самые близкие к заданным:

In [3]:
model = api.load("word2vec-ruscorpora-300")

[==================================================] 100.0% 198.8/198.8MB downloaded


In [4]:
model.most_similar('человек_NOUN', topn=5) #topn — сколько ближайших слов мы хотим получить

[('женщина_NOUN', 0.5500056743621826),
 ('мужчина_NOUN', 0.5161216855049133),
 ('человеческий_ADJ', 0.5005477666854858),
 ('идолопоклонствовать_VERB', 0.48388850688934326),
 ('высокопорядочный_ADJ', 0.4818764925003052)]

In [5]:
model.most_similar('счастливый_ADJ', topn=15)

[('несчастливый_ADJ', 0.677182674407959),
 ('счастие_NOUN', 0.6733449697494507),
 ('счастливо_ADV', 0.6329190731048584),
 ('радостный_ADJ', 0.6159152388572693),
 ('беззаботный_ADJ', 0.5699828267097473),
 ('пресчастливый_ADJ', 0.551605224609375),
 ('радоваться_VERB', 0.5468602180480957),
 ('довольный_ADJ', 0.5460575819015503),
 ('несчастный_ADJ', 0.5273663401603699),
 ('благополучный_ADJ', 0.526889443397522),
 ('веселый_ADJ', 0.5250555276870728),
 ('счастливец_NOUN', 0.5245419144630432),
 ('безмятежный_ADJ', 0.5210704803466797),
 ('беспечный_ADJ', 0.519269585609436),
 ('многолюбить_VERB', 0.5067941546440125)]

Обратите внимание, что все слова помечены частями речи.

Попробуйте найти косинусное расстояние между векторными представлениями слов "кошка" и "собака", "дом" и "дерево", "радостный" и "грустный" (подсказка: вам НЕ нужно руками писать формулу косинусного расстояния).

In [0]:
#YOUR CODE HERE

Заметим, что слова, обозначающие объекты разной природы, отличаются гораздо сильнее, чем антонимичные по смыслу слова. Как вы думаете, почему?

Проверьте, работают ли с векторными представлениями слов арифметические операции: "король" - "мужчина" + "женщина" = ?

In [0]:
#YOUR CODE HERE

Однако результат зависит от использованного текстового корпуса, и такую предобученную модель вряд ли можно будет использовать, например, для анализа записей из Twitter. Поэтому может возникнуть необходимость обучить модель самостоятельно.

В качестве примера рассмотрим "игрушечный" датасет, который есть в gensim.

In [0]:
from gensim.test.utils import common_texts

for text in common_texts:
  print(text)

model_w2v_toy = Word2Vec(common_texts, size=20, min_count=1)
model_w2v_toy.wv.most_similar(positive='human', topn=3)

Однако для настоящего обучения модели нужен текст очень большого размера. Возьмём, например, Библию (потому что почему бы и нет). Обратите внимание, что перед обучением текст нужно предобработать (выбросить пунктуацию и стоп-слова).

In [0]:
!pip install pymorphy2

In [0]:
import urllib.request
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
#эта ячейка будет работать ДОЛГО
nltk.download('stopwords')
morph = pymorphy2.MorphAnalyzer()
regex = re.compile('[^а-яА-Я ё\-]')
stop = stopwords.words('russian')
# + ['сказать', 'и', 'твой']

data = urllib.request.urlopen('https://raw.githubusercontent.com/somethingneverending/nlp-files/master/Bible.txt')
X = []
for line in data:
  cur = regex.sub('',line.decode('utf-8')).split('\n')[0].split(' ')
  for word in cur:
    normal_word = morph.parse(word.lower())[0].normal_form
    if word != '' and normal_word not in stop:
      X.append(normal_word)

In [0]:
X[:20]

In [0]:
model_w2v = Word2Vec([X], size=300, window=5, min_count=1)

Посмотрите на синонимы тех же слов, которые мы рассматривали для предобученной модели.

In [0]:
#YOUR CODE HERE

...Кажется, что-то изменилось.

Визуализируем 55 наиболее встречающихся слов:

In [0]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

vocab = [(model_w2v.wv.vocab[x].count, x) for x in model_w2v.wv.vocab]
vocab.sort(reverse=True)
vocab = vocab[:55]

def visualize(vocab):
  all_vocab = []
  for i in range(len(vocab)):
    all_vocab = all_vocab + vocab[i]
  emb_tuple = tuple([model_w2v.wv[word] for word in all_vocab])
  X_vis = np.vstack(emb_tuple)

  model_tsne = TSNE(n_components=2, random_state=0)
  np.set_printoptions(suppress=True)

  X_tsne = model_tsne.fit_transform(X_vis)
  cur = 0

  for part in vocab:
    word_labels = [word for word in part]
    plt.scatter(X_tsne[cur:cur + len(part), 0], X_tsne[cur:cur + len(part), 1])
    for i, word in enumerate(word_labels):
        plt.annotate(word, (X_tsne[cur + i, 0], X_tsne[cur + i, 1]))
    cur += len(part)
  plt.show()

vocab = [tmp37[1] for tmp37 in vocab]
visualize([vocab])

Сделайте то же самое, например, для тридцати слов, ближайшим к какому-нибудь слову; для двадцати ближайших и двадцати самых дальних.

In [0]:
#YOUR CODE HERE

In [0]:
#YOUR CODE HERE

То же самое можете сделать самостоятельно и с предобученной моделью.